In [1]:
import requests
import os
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import optimization_v1
import googlemaps
from dotenv import load_dotenv
import pytz
from datetime import datetime, timezone, timedelta
import pandas
import numpy as np
import io


#assumptions:
# optimal time 7:15 to 9:15
# latest delivery time 9:30


load_dotenv()
proxy_address = "http://127.0.0.1:7890"
# Set the environment variables to specify the proxy
os.environ["HTTP_PROXY"] = proxy_address
os.environ["HTTPS_PROXY"] = proxy_address
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kaicheng/ProjectsFormal/wheelTrans/secrets/key.json"
os.environ["PROJECT_ID"] = "elderlyhometransportation"
googleMapKey = os.getenv("GOOGLE_MAP_API_KEY")

ROOT_PATH = os.path.dirname(os.path.abspath(''))
ROOT_PATH


'/Users/kaicheng/ProjectsFormal/wheelTrans/src'

In [2]:
weekday = 'M' # 'M', 'T', 'W', 'R', 'F'

def handleInvalidAddress(df):
    print ("toDo")

df = pandas.read_excel("resources/TransportList.xlsx")
validDf = df[df['Address'].notna()]
handleInvalidAddress(df)
attendingAMdf = df[df[weekday].notna() & (df['Trans Method'].str.contains('am', case=False))]
cleanDf = attendingAMdf.loc[:, ['MR #','Address','Trans Method', 'M', 'T', 'W', 'R', 'F', 'Notes']]
#dev short
# cleanDf = cleanDf[:10]
cleanDf

toDo


,MR #,Address,Trans Method,M,T,W,R,F,Notes
0,001M,"3843 Maxson Road #226 El Monte, CA 91732",ampm,X,NaN,X,NaN,X,8:00+
1,002F,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+
2,003M,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+
5,007M,"2417 Angelus Ave #306 6266880563 Rosemead, CA ...",ampm,X,NaN,X,NaN,X,NaN
6,009F,"201 1/2 W Newmark Ave Monterey Park, CA 91755",ampm,X,NaN,X,NaN,X,NaN
...,...,...,...,...,...,...,...,...,...
160,167M,"7529 Teresa Ave Rosemead, CA",ampm,X,X,X,X,X,NaN
162,169F,"1206 S 3rd Ave Arcadia,CA",ampm,X,X,X,X,X,NaN
165,172F,"6464 N Vista St San Gabriel, CA ...",ampm,X,X,NaN,X,X,NaN
166,173F,"2511 Hagen Dr, Alhambra, CA,",am,X,X,X,X,X,NaN


In [3]:
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
columns = cleanDf.columns
geocodedDf = pandas.DataFrame(columns=columns)
invalidGeoCodeDf = pandas.DataFrame(columns=columns)

def get_direction(origin, destination, waypoints):
    waypoints_str = "|".join(["optimize:true"] + waypoints)
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    google_map_api_key = os.getenv("GOOGLE_MAP_API_KEY")

    params = {
        "destination": destination,
        "origin": origin,
        "waypoints": waypoints_str,
        "key": google_map_api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


def get_geocoding(gmaps, address):
    result = gmaps.geocode(address)
    if not result or len(result) != 1:
        print(f"Failed to get geocoding for {address}")
        return None
    location = result[0]['geometry']['location']
    lat = location["lat"]
    lng = location["lng"]
    return [lat, lng]


origin = "8408 Garvey Ave. #101 Rosemead, CA 91770"

for ind, row in cleanDf.iterrows():
    latlng = get_geocoding(gmaps,  row['Address'])
    if latlng:
        row['lat'] = latlng[0]
        row['lng'] = latlng[1]
        geocodedDf = pandas.concat([geocodedDf, pandas.DataFrame([row])], ignore_index=True, axis=0)
    else:
        print(f"Failed to get geocoding for {row['Address']}")
        invalidGeoCodeDf = pandas.concat([invalidGeoCodeDf, pandas.DataFrame([row])], ignore_index=True, axis=0)
geocodedDf

Failed to get geocoding for nan
Failed to get geocoding for nan


,MR #,Address,Trans Method,M,T,W,R,F,Notes,lat,lng
0,001M,"3843 Maxson Road #226 El Monte, CA 91732",ampm,X,NaN,X,NaN,X,8:00+,34.072393,-118.010237
1,002F,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+,34.065167,-118.123730
2,003M,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+,34.065167,-118.123730
3,007M,"2417 Angelus Ave #306 6266880563 Rosemead, CA ...",ampm,X,NaN,X,NaN,X,NaN,34.056449,-118.087469
4,009F,"201 1/2 W Newmark Ave Monterey Park, CA 91755",ampm,X,NaN,X,NaN,X,NaN,34.060019,-118.124452
...,...,...,...,...,...,...,...,...,...,...,...
58,166M,"275 Cordova St Apt 606 Pasadena, CA",ampm,X,X,X,X,X,NaN,34.142452,-118.144862
59,167M,"7529 Teresa Ave Rosemead, CA",ampm,X,X,X,X,X,NaN,34.049553,-118.102412
60,169F,"1206 S 3rd Ave Arcadia,CA",ampm,X,X,X,X,X,NaN,34.126736,-118.023137
61,172F,"6464 N Vista St San Gabriel, CA ...",ampm,X,X,NaN,X,X,NaN,34.116944,-118.083416


In [4]:
invalidGeoCodeDf.to_csv('output/invalidAddress.csv', index=False)
invalidGeoCodeDf

,MR #,Address,Trans Method,M,T,W,R,F,Notes
0,178F,NaN,ampm,X,X,X,X,X,NaN


In [5]:
from resources.vehicles import vehicles
import json  
import re

def check_time_format(time_str):
    pattern = r'^\d{1,2}:\d{2}[+-]$'
    if re.match(pattern, time_str):
        return True
    else:
        return False
shipments = []
for ind, row in geocodedDf.iterrows():
    shipment = {
        "loadDemands": {
            "weight": {
                "amount": "1"
            }
        },
        "pickups": [
            {
                "arrivalLocation": {
                    "latitude": row['lat'],
                    "longitude": row['lng']
                },
                "duration": "60s",
            }
        ],
        "deliveries": [
            {
                "arrival_location": {
                    "latitude": 34.0623483,
                    "longitude": -118.0859541
                },
                "duration": "10s",
            }
        ]
    }

    note = row['Notes'] if row['Notes'] is not np.nan else ''
    isValidTime = check_time_format(note)
    if isValidTime:
        pickupTime = note.strip('+-')
        if note.endswith('-'):
            startTime = "2024-03-08T7:15:00Z"
            endTime = f"2024-03-08T{pickupTime}:00Z"
        elif note.endswith('+'):
            startTime = f"2024-03-08T{pickupTime}:00Z"
            endTime = "2024-03-08T9:30:00Z"
        shipment['pickups'][0]["timeWindows"] = [{
                "startTime": startTime,
                "endTime": endTime
            }]
        print(f"shipment {ind} start time {startTime} end time {endTime}")
    else:
        if note:  print(f"Invalid time format in the note {note} for {row['MR #']}")
    shipments.append(shipment)

requestDict = {
    "parent": "projects/elderlyhometransportation",
    "model": {
        "shipments": shipments,
        "vehicles": vehicles,
        "global_start_time":"2024-03-08T7:15:00Z",
        "global_end_time":"2024-03-08T9:30:00Z",
        "global_duration_cost_per_hour": "60",
    },
    "populatePolylines": True,
    # "searchMode":2
}
requestJson = json.dumps(requestDict)


shipment 0 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z
shipment 1 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z
shipment 2 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z
shipment 11 start time 2024-03-08T7:15:00Z end time 2024-03-08T7:40:00Z
shipment 12 start time 2024-03-08T7:15:00Z end time 2024-03-08T7:40:00Z
shipment 18 start time 2024-03-08T7:15:00Z end time 2024-03-08T8:00:00Z
shipment 19 start time 2024-03-08T7:15:00Z end time 2024-03-08T8:00:00Z
shipment 27 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z


In [8]:
fleet_routing_client = optimization_v1.FleetRoutingClient()
fleetOptimizationRequest = optimization_v1.OptimizeToursRequest.from_json(requestJson)

fleetOptimizationResponse = fleet_routing_client.optimize_tours(
    fleetOptimizationRequest, timeout=100,
)
fleetOptimizationResponse

routes {
  vehicle_start_time {
    seconds: 1709882100
  }
  vehicle_end_time {
    seconds: 1709888708
  }
  visits {
    shipment_index: 62
    is_pickup: true
    start_time {
      seconds: 1709882686
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
    }
    arrival_loads {
      type_: "weight"
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 8
    is_pickup: true
    start_time {
      seconds: 1709883356
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 439
    }
    arrival_loads {
      type_: "weight"
      value: 1
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 58
    is_pickup: true
    start_time {
      seconds: 1709884560
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 955
    }


In [15]:
import polyline

optimizedList=[]
for vehicleRoute in fleetOptimizationResponse.routes:
    points = vehicleRoute.route_polyline.points
    startingMarker = f'34.0623483,-118.0859541'
    vehicleIndex = vehicleRoute.vehicle_index or 0

    #trim duplicate dropoff
    prev_isPickup = None
    visits = []
    for element in vehicleRoute.visits:
        isPickup = element.is_pickup
        if isPickup or isPickup != prev_isPickup:
            visits.append(element)
        prev_isPickup = isPickup
    tripNumber = 0        
    # write result to df
    for index, visit in enumerate(visits):
        #increment trip number upon dropoff
        if not visit.is_pickup: #dropoff
            data = {
                "MR #": "",
                "Arrival Time": visit.start_time,
                "vehicle": vehicleIndex,
                "order": index,
                "routePolyline": points,
                "tripNumber": tripNumber,
                "Address": "→ Return to 8408 Garvey Ave. #101 Rosemead, CA 91770",
                "lat":'34.0623483',
                "lng": '-118.0859541'
            }
            row = pandas.Series(data)
            optimizedList.append(row)
            tripNumber += 1
            continue

        shipmentId = visit.shipment_index or 0
        row = geocodedDf.loc[shipmentId].copy()
        row["Arrival Time"]= visit.start_time
        row["vehicle"]= vehicleIndex
        row["order"]= index
        row["routePolyline"]= points
        row["tripNumber"]= tripNumber
        optimizedList.append(row)
    
optimizedDf = pandas.DataFrame(optimizedList)
optimizedDf['Address'].values

array(['2511 Hagen Dr, Alhambra, CA, ',
       '4000 E. Fairmount St. #607  Los Angeles, CA 90063',
       '275 Cordova St Apt 606 Pasadena, CA',
       '275 Cordova St Apt 606 Pasadena, CA',
       '111 N Chapel Ave Apt 805 Alhambra, CA 91801',
       '111n Chapel Ave Apt 805 Alhambra, CA 91801',
       '→ Return to 8408 Garvey Ave. #101 Rosemead, CA 91770',
       '1742 Eckhart Ave Rosemead, CA',
       '140 N Montebello Blvd Apt 208, Montebello, CA, 90640  ',
       '140n Montebello Blvd 325, Monterey Park, CA 91755           ',
       '140 N Montebello Blvd Apt 208 Montebello, CA 90640',
       '1315 W Beverly Blvd #324 Montebello, CA 90640',
       '140 N Montebello Blvd 511 Montebello, CA 90640',
       '1540 Potrero Grande Ave  Rosemead, CA 91770',
       '→ Return to 8408 Garvey Ave. #101 Rosemead, CA 91770',
       '9106 Newby Ave Rosemead, CA 91770',
       '9423 Steele Street  Rosemead, CA 91770',
       '9527 Guess Street  Rosemead, CA 91770',
       '8308 Keim St Rosemead,

In [32]:
import polyline
from PIL import Image, ImageDraw, ImageFont, ImageOps
from resources.vehicles import vehicles


def drawToImage(byteImage, filename, waypoints):
    image = Image.open(io.BytesIO(byteImage))
    image = image.convert("RGB")
    padded_image = ImageOps.expand(image, (800, 0,0,500), fill="white")
    draw = ImageDraw.Draw(padded_image)
    text ='\n'.join(waypoints)
    font = ImageFont.load_default()
    font_size = 15
    font = font.font_variant(size=font_size)
    text_position = (10, 10)
    text_color = (0, 0, 0)
    draw.text(text_position, text, fill=text_color, font=font)

    padded_image.save(f"output/{filename}.png")

def generate_static_map(origin, destination, routePolyline, waypoints,filename):
    startingMarker = f'34.0623483,-118.0859541'
    # Add markers for each waypoint with labels
    markers = []
    for i, waypoint in enumerate(waypoints):
        markers.append(f"markers=label:{i}|color:red|{waypoint[0]},{waypoint[1]}")
    markers_str = '&'.join(markers)
    static_map_url = f'https://maps.googleapis.com/maps/api/staticmap?size=1200x800&path=enc:{routePolyline}&{markers_str}&markers=color:blue|{startingMarker}&key={googleMapKey}'
    print (static_map_url)
    # Download the map image
    map_image = requests.get(static_map_url)
    return map_image.content
def create_google_maps_url(waypoints):
    base_url = "https://www.google.com/maps/dir/?api=1"
    origin = "origin=34.0623483,-118.0859541"
    destination = "destination=34.0623483,-118.0859541"
    parameters = "|".join([f"{waypoint[0]},{waypoint[1]}" for waypoint in waypoints])
    return f"{base_url}&{origin}&waypoints={parameters}&{destination}"

origin = (34.0623483, -118.0859541)  # Los Angeles, CA
destination = (34.0623483, -118.0859541)  # Los Angeles, CA
vehiclesIds = optimizedDf['vehicle'].unique()
for vehicleId in vehiclesIds:
    number_of_seats = vehicles[vehicleId]['loadLimits']['weight']['maxLoad']
    sortedDf = optimizedDf[optimizedDf['vehicle'] == vehicleId].sort_values(by='order')
    tripIds = np.sort(sortedDf['tripNumber'].unique())
    routePolyline = sortedDf['routePolyline'].values[0]
    NOfPickups = 0
    with open(f'output/waypointInfo{vehicleId}.csv', 'w') as f:
        f.write(f'Vehicle {vehicleId+1}, {number_of_seats} seats.' + '\n')
    for tripId in tripIds:
        tripDf = sortedDf[sortedDf['tripNumber'] == tripId]
        waypoints = [[lat, lon] for lat, lon in tripDf[['lat', 'lng']].values]
        navigationUrl = create_google_maps_url(waypoints)
        # byteImage=generate_static_map(origin=origin, destination=destination, routePolyline=routePolyline,waypoints=waypoints, filename=f'map_with_route{vehicleId}.png' )
        dfMeta = pandas.DataFrame([[],[f"Trip {tripId}", 'Google map Url', navigationUrl]])
        dfMeta.to_csv(f'output/waypointInfo{vehicleId}.csv',header=False, index=False, mode='a')
        tripDf['Arrival Time'] = tripDf['Arrival Time'].dt.strftime('%H:%M')
        tripDf['order'] = tripDf['order'] + 1
        tripDf[['order', 'MR #', 'Address', 'Arrival Time', 'Notes']].to_csv(f'output/waypointInfo{vehicleId}.csv', header=True, index=False, mode='a')
        #row count
        NOfPickups+=tripDf[tripDf['MR #']!=''].shape[0]

        
        # # save to image
        # with open(f"output/{vehicleId}.png", "wb") as f:
        #     f.write(byteImage)
        # #draw overlay
        # header=[f'vehicle Id:{vehicleId +1}       number of seats: {number_of_seats}\n\n\n\n\n'
        #         ,'order        '+'MR #        ' +' Address                                                          '+    'timeOfArrival        '+    'notes\n'] 
        # displayWaypointInfo=header+[f"{order}        {MR}        {Address if Address is not np.nan else ' '}            {arrivalTime}            {Notes if Notes is not np.nan else ' '}" for order, MR, Address, arrivalTime, Notes in tripDf[['order', 'MR #', 'Address', 'arrival time', 'Notes']].values]
        # drawToImage(byteImage, f'vehicle_{vehicleId+1}_seat_{number_of_seats}_tripId{tripId+1}', displayWaypointInfo)
    print(f"Vehicle {vehicleId+1} has {NOfPickups} passengers")
dfMeta

Vehicle 1 has 13 passengers
Vehicle 2 has 14 passengers
Vehicle 3 has 22 passengers
Vehicle 4 has 14 passengers


/var/folders/tt/gb1w4qx136lggdtv_zwlj_vm0000gp/T/ipykernel_46029/1576157409.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tripDf['Arrival Time'] = tripDf['Arrival Time'].dt.strftime('%H:%M')
/var/folders/tt/gb1w4qx136lggdtv_zwlj_vm0000gp/T/ipykernel_46029/1576157409.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tripDf['order'] = tripDf['order'] + 1
/var/folders/tt/gb1w4qx136lggdtv_zwlj_vm0000gp/T/ipykernel_46029/1576157409.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,0,1,2
0,None,None,None
1,Trip 3,Google map Url,https://www.google.com/maps/dir/?api=1&origin=...
